In [2]:
import numpy as np
import pandas

1.Загрузите выборку из файла titanic.csv с помощью пакета Pandas.

In [5]:
df = pandas.read_csv('titanic.csv', index_col='PassengerId')
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


1.1.Какое количество мужчин и женщин ехало на корабле? В качестве ответа приведите два числа через пробел.

In [6]:
count_sex=df['Sex'].value_counts()
print(count_sex['male'],count_sex['female'])

577 314


1.2.Какой части пассажиров удалось выжить? Посчитайте долю выживших пассажиров. Ответ приведите в процентах (число в интервале от 0 до 100, знак процента не нужен), округлив до двух знаков.

In [7]:
count_survived = df['Survived'].value_counts()
print(round(100 * count_survived[1] / len(df), 2))

38.38


1.3.Какую долю пассажиры первого класса составляли среди всех пассажиров? Ответ приведите в процентах (число в интервале от 0 до 100, знак процента не нужен), округлив до двух знаков.

In [8]:
count_pclass = df['Pclass'].value_counts()
print(round(100 * count_pclass[1] / count_pclass.sum(),2))

24.24


1.4.Какого возраста были пассажиры? Посчитайте среднее и медиану возраста пассажиров. В качестве ответа приведите два числа через пробел.

In [9]:
age_info= df['Age'].dropna()
print(round(age_info.mean(),2),round(age_info.median(),2))

29.7 28.0


1.5.Коррелируют ли число братьев/сестер/супругов с числом родителей/детей? Посчитайте корреляцию Пирсона между признаками SibSp и Parch.

In [10]:
print(round(df['SibSp'].corr(df['Parch']), 2))

0.41


1.6.Какое самое популярное женское имя на корабле? Извлеките из полного имени пассажира (колонка Name) его личное имя (First Name). Это задание — типичный пример того, с чем сталкивается специалист по анализу данных. Данные очень разнородные и шумные, но из них требуется извлечь необходимую информацию. Попробуйте вручную разобрать несколько значений столбца Name и выработать правило для извлечения имен, а также разделения их на женские и мужские.

In [15]:
import re
def extract_name(name):
    e_n = re.search('^[^,]+, (.*)', name)
    if e_n:
        name = e_n.group(1)

    e_n = re.search('\(([^)]+)\)', name)
    if e_n:
        name = e_n.group(1)

    name = re.sub('(Miss\. |Mrs\. |Ms\. )', '', name)
    return name


names = df[df['Sex'] == 'female']['Name'].map(extract_name)
print(names.value_counts().head(1).index.values[0])

Mary


2.Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).

In [16]:
x_labels = ['Pclass', 'Fare', 'Age', 'Sex']
df_new = df.loc[:, x_labels]
df_new.head()

,Pclass,Fare,Age,Sex
PassengerId,,,,
1,3,7.2500,22.0,male
2,1,71.2833,38.0,female
3,3,7.9250,26.0,female
4,1,53.1000,35.0,female
5,3,8.0500,35.0,male


2.1. Обратите внимание, что признак Sex имеет строковые значения

In [17]:
df_new['Sex'] = df_new['Sex'].map(lambda sex: 1 if sex == 'male' else 0)

2.2. Выделите целевую переменную — она записана в столбце Survived

In [18]:
y= df['Survived']

2.3. В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен их возраст. Такие записи при чтении их в pandas принимают значение nan. Найдите все объекты, у которых есть пропущенные признаки, и удалите их из выборки.

In [19]:
df_new = df_new.dropna()
y = y[df_new.index.values]

2.4.Обучите решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию (речь идет о параметрах конструктора DecisionTreeСlassifier).

In [20]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=241)
dtc.fit(np.array(df_new.values), np.array(y.values))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=241, splitter='best')

2.5.	Вычислите важности признаков и найдите два признака с наибольшей важностью. Их названия будут ответами для данной задачи (в качестве ответа укажите названия признаков через запятую или пробел, порядок не важен).

In [21]:
importance = pandas.Series(dtc.feature_importances_, index=x_labels)
print(' '.join(importance.sort_values(ascending=False).head(2).index.values))

Fare Sex
